<a href="https://colab.research.google.com/github/thaisrezendeb/ufes_machineLearning/blob/main/L2_Exercicio2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**2) Aplique o Naive Bayes sobre a base de dados Balance Scale (disponível em http://archive.ics.uci.edu/ml/) utilizando o procedimento de Hold-Out dez vezes, na proporção de 75% de amostras de treino e 25% de teste. Obtenha a acurácia média e o desvio padrão da acurácia. Realize os experimentos:**

**a) Considerando uma distribuição Gaussiana dos atributos;**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

hold = 10
size_train = 0.75

uri = "https://archive.ics.uci.edu/ml/machine-learning-databases/balance-scale/balance-scale.data"
balance = pd.read_csv(uri, sep=",", header=None)

def prob_gaussian(med, dp, x):
  
  pgauss = (1 / \
            (np.sqrt(2 * np.pi) * dp)) * np.exp(-((x-med)**2 / (2 * dp**2 )))

  return pgauss

In [ ]:
acuracia = np.zeros((hold))

for i in range(0,hold):
  acertos = 0

  #Seleção aleatória das amostras
  #np.random.seed(7)
  sel_train = np.random.choice(balance.shape[0], round(balance.shape[0]*size_train), replace=False)
  balance_train = balance.iloc[sel_train]
  balance_test = balance.drop(sel_train)

  #Cálculo de Naive Bayes - Classes: L, B, R
  # Prioridades a priori
  bt_l = balance_train[balance_train[0] == 'L']
  p_l = (bt_l.shape[0] / balance_train.shape[0])

  bt_b = balance_train[balance_train[0] == 'B']
  p_b = (bt_b.shape[0] / balance_train.shape[0])

  bt_r = balance_train[balance_train[0] == 'R']
  p_r = (bt_r.shape[0] / balance_train.shape[0])

  # Matriz de probabilidades gaussianas a posteriori para cada valor de atributo/classe 
  # Cada linha corresponde a uma classe
  # (todos os quatro atributos variam de 1 a 5)
  # Probabilidades de cada atributo para cada classe - P(Xn=val|C=c) 
  # Cada coluna corresponde a um valor de atributo (encadeados)
  # Última posição de cada linha corresponde à probabilidade à priori da classe
  p_x = np.zeros((3,21))
  
  #Probabilidades da classe L
  med_l = np.array(bt_l.mean()) #Média de cada atributo
  dp_l = np.array(bt_l.std()) #Desvio padrão de cada atributo
  p_x[0,20] = p_l
  for n in range(0,4): #atributos
    for val in range(0,5): #valores
      p_x[0,n*5+val] = prob_gaussian(med_l[n], dp_l[n], val+1)
  
  #Probabilidades da classe B
  med_b = np.array(bt_b.mean()) #Média de cada atributo
  dp_b = np.array(bt_b.std()) #Desvio padrão de cada atributo
  p_x[1,20] = p_b
  for n in range(0,4): #atributos
    for val in range(0,5): #valores
      p_x[1,n*5+val] = prob_gaussian(med_b[n], dp_b[n], val+1)

  #Probabilidades da classe R
  med_r = np.array(bt_r.mean()) #Média de cada atributo
  dp_r = np.array(bt_r.std()) #Desvio padrão de cada atributo
  p_x[2,20] = p_r
  for n in range(0,4): #atributos
    for val in range(0,5): #valores
      p_x[2,n*5+val] = prob_gaussian(med_r[n], dp_r[n], val+1)

  # Calculada a matriz de probabilidades sobre as amostras de treinamento,
  # calcula-se a acurácia do algoritmo
  for w,row in balance_test.iterrows():
    max_p = 0
    #Soma os percentuais de cada atributo
    for j in range(0,3): #classes
      total_p = p_x[j,20]  #probabilidade a priori
      for k in range(0,4): #atributos
        total_p = total_p * p_x[j,k*5+row[k+1]-1] 
    
      if total_p > max_p:
        max_p = total_p
        new_class = {0: "L", 1: "B", 2: "R"}[j]

    #Verificação de acerto
    #print(row[0], new_class)
    if row[0] == new_class:
      acertos = acertos + 1

  acuracia[i] = acertos / balance_test.shape[0]

#Finalização dos cálculos
#------------------------
print("Acurácia média usando distribuição gaussiana: {:.2f}%".format(acuracia.mean()*100))
print("Desvio padrão acurácia: {:.3f}".format(acuracia.std()))

Acurácia média usando distribuição gaussiana: 90.83%
Desvio padrão acurácia: 0.021


**b) Discretizando os valores (em 5 bins cada atributo);**

Como o valores dos atributos já são discretizados e com valores de 1 a 5, não há necessidade de criar os bins para os atributos. Porém, caso houvesse necessidade, poderia-se escolher a seguinte regra de discretização:

* ]0,1] = 1
* ]1,2] = 2
* ]2,3] = 3
* ]3,4] = 4
* ]4,5] = 5


In [ ]:
#Discretização dos atributos em código
#for n in range(0,5):
#  balance[(balance > n) & (balance <= n+1)] = n+1

acuracia = np.zeros((hold))

for i in range(0,hold):
  acertos = 0

  #Seleção aleatória das amostras
  sel_train = np.random.choice(balance.shape[0], round(balance.shape[0]*size_train), replace=False)
  balance_train = balance.iloc[sel_train]
  balance_test = balance.drop(sel_train)

  #Cálculo de Naive Bayes - Classes: L, B, R
  # Prioridades a priori
  bt_l = balance_train[balance_train[0] == 'L']
  p_l = (bt_l.shape[0] / balance_train.shape[0])

  bt_b = balance_train[balance_train[0] == 'B']
  p_b = (bt_b.shape[0] / balance_train.shape[0])

  bt_r = balance_train[balance_train[0] == 'R']
  p_r = (bt_r.shape[0] / balance_train.shape[0])

  # Matriz de probabilidades gaussianas a posteriori para cada valor de atributo/classe 
  # Cada linha corresponde a uma classe
  # (todos os quatro atributos variam de 1 a 5)
  # Probabilidades de cada atributo para cada classe - P(Xn=val|C=c) 
  # Cada coluna corresponde a um valor de atributo (encadeados)
  # Última posição de cada linha corresponde à probabilidade à priori da classe
  p_x = np.zeros((3,21))
  
  #Probabilidades condicionais da classe L
  p_x[0,20] = p_l
  for n in range(0,4): #atributos
    for val in range(0,5): #valores
      p_x[0,n*5+val] = (bt_l[bt_l[n+1] == val+1].shape[0] / bt_l.shape[0])
  
  #Probabilidades condicionais da classe B
  p_x[1,20] = p_b
  for n in range(0,4): #atributos
    for val in range(0,5): #valores
      p_x[1,n*5+val] = (bt_b[bt_b[n+1] == val+1].shape[0] / bt_b.shape[0])
  
  #Probabilidades condicionais da classe R
  p_x[2,20] = p_r
  for n in range(0,4): #atributos
    for val in range(0,5): #valores
      p_x[2,n*5+val] = (bt_r[bt_r[n+1] == val+1].shape[0] / bt_r.shape[0])
  
  # Calculada a matriz de probabilidades sobre as amostras de treinamento,
  # calcula-se a acurácia do algoritmo
  for w,row in balance_test.iterrows():
    max_p = 0
    #Soma os percentuais de cada atributo
    for j in range(0,3): #classes
      total_p = p_x[j,20]  #probabilidade a priori
      for k in range(0,4): #atributos
        total_p = total_p * p_x[j,k*5+row[k+1]-1] 
    
      if total_p > max_p:
        max_p = total_p
        new_class = {0: "L", 1: "B", 2: "R"}[j]

    #Verificação de acerto
    #print(row[0], new_class)
    if row[0] == new_class:
      acertos = acertos + 1

  acuracia[i] = acertos / balance_test.shape[0]

#Finalização dos cálculos
#------------------------
print("Acurácia média: {:.2f}%".format(acuracia.mean()*100))
print("Desvio padrão acurácia: {:.3f}".format(acuracia.std()))

Acurácia média: 89.23%
Desvio padrão acurácia: 0.021


**c) Discretize os valore da mesma forma que em b) usando a suavização de Laplace.**

In [ ]:
#Discretização dos atributos em código
#for n in range(0,5):
#  balance[(balance > n) & (balance <= n+1)] = n+1

acuracia = np.zeros((hold))

for i in range(0,hold):
  acertos = 0

  #Seleção aleatória das amostras
  sel_train = np.random.choice(balance.shape[0], round(balance.shape[0]*size_train), replace=False)
  balance_train = balance.iloc[sel_train]
  balance_test = balance.drop(sel_train)

  #Cálculo de Naive Bayes - Classes: L, B, R
  # Prioridades a priori com suavização de Laplace
  bt_l = balance_train[balance_train[0] == 'L']
  p_l = ((bt_l.shape[0]+1) / (balance_train.shape[0]+3))

  bt_b = balance_train[balance_train[0] == 'B']
  p_b = ((bt_b.shape[0]+1) / (balance_train.shape[0]+3))

  bt_r = balance_train[balance_train[0] == 'R']
  p_r = ((bt_r.shape[0]+1) / (balance_train.shape[0]+3))

  # Matriz de probabilidades gaussianas a posteriori para cada valor de atributo/classe 
  # Cada linha corresponde a uma classe
  # (todos os quatro atributos variam de 1 a 5)
  # Probabilidades de cada atributo para cada classe - P(Xn=val|C=c) 
  # Cada coluna corresponde a um valor de atributo (encadeados)
  # Última posição de cada linha corresponde à probabilidade à priori da classe
  p_x = np.zeros((3,21))
  
  #Probabilidades condicionais da classe L com suavização por Laplace
  p_x[0,20] = p_l
  for n in range(0,4): #atributos
    for val in range(0,5): #valores
      p_x[0,n*5+val] = ((bt_l[bt_l[n+1] == val+1].shape[0]+1) / (bt_l.shape[0]+5))
  
  #Probabilidades condicionais da classe B com suavização por Laplace
  p_x[1,20] = p_b
  for n in range(0,4): #atributos
    for val in range(0,5): #valores
      p_x[1,n*5+val] = ((bt_b[bt_b[n+1] == val+1].shape[0]+1) / (bt_b.shape[0]+5))
  
  #Probabilidades condicionais da classe R com suavização por Laplace
  p_x[2,20] = p_r
  for n in range(0,4): #atributos
    for val in range(0,5): #valores
      p_x[2,n*5+val] = ((bt_r[bt_r[n+1] == val+1].shape[0]+1) / (bt_r.shape[0]+5))
  
  # Calculada a matriz de probabilidades sobre as amostras de treinamento,
  # calcula-se a acurácia do algoritmo
  for w,row in balance_test.iterrows():
    max_p = 0
    #Soma os percentuais de cada atributo
    for j in range(0,3): #classes
      total_p = p_x[j,20]  #probabilidade a priori
      for k in range(0,4): #atributos
        total_p = total_p * p_x[j,k*5+row[k+1]-1] 
    
      if total_p > max_p:
        max_p = total_p
        new_class = {0: "L", 1: "B", 2: "R"}[j]

    #Verificação de acerto
    #print(row[0], new_class)
    if row[0] == new_class:
      acertos = acertos + 1

  acuracia[i] = acertos / balance_test.shape[0]

#Finalização dos cálculos
#------------------------
print("Acurácia média com suavização por Laplace: {:.2f}%".format(acuracia.mean()*100))
print("Desvio padrão acurácia: {:.3f}".format(acuracia.std()))

Acurácia média com suavização por Laplace: 90.58%
Desvio padrão acurácia: 0.023
